In [38]:
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import CTransformers

import os
from dotenv import load_dotenv

In [39]:
load_dotenv()

pinecone_api_key = os.getenv("PINECONE_API_KEY")
pinecone_api_env = 'gcp-starter'

In [40]:
# Etract data from PDF
def load_pdf(data):
   loader =  DirectoryLoader(path=data, 
                            glob='*.pdf',
                            loader_cls=PyPDFLoader)
   documents = loader.load()

   return documents

In [41]:
extracted_data = load_pdf('data/')
extracted_data

In [42]:
# Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, 
                                                chunk_overlap=20)
    text_chunks = text_splitter.split_documents(documents=extracted_data)
    
    return text_chunks

In [43]:
text_chunks = text_split(extracted_data)
len(text_chunks)
# text_chunks

10484


In [44]:
# Embedding model
def download_embeddings_model():
    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings



In [45]:
embeddings = download_embeddings_model()

/var/folders/l5/nfl8tw2j4jb7l_5plk_dxddr0000gn/T/ipykernel_37628/4235689176.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
/opt/homebrew/Caskroom/miniconda/base/envs/med-ast/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/homebrew/Caskroom/miniconda/base/envs/med-ast/lib/python3.11/site-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'cached_download' (fro

In [46]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [47]:
query_result

[-0.03447728604078293,
 0.031023267656564713,
 0.006734997034072876,
 0.02610897831618786,
 -0.0393621064722538,
 -0.16030247509479523,
 0.066924087703228,
 -0.00644137617200613,
 -0.04745044559240341,
 0.014758842997252941,
 0.07087539881467819,
 0.05552750080823898,
 0.019193340092897415,
 -0.02625138871371746,
 -0.01010960340499878,
 -0.026940476149320602,
 0.022307490929961205,
 -0.022226663306355476,
 -0.149692565202713,
 -0.01749301701784134,
 0.007676353678107262,
 0.05435230955481529,
 0.003254495793953538,
 0.03172589838504791,
 -0.08462144434452057,
 -0.0294061116874218,
 0.05159566551446915,
 0.048124078661203384,
 -0.0033147772774100304,
 -0.058279260993003845,
 0.04196932166814804,
 0.022210663184523582,
 0.128188818693161,
 -0.022338958457112312,
 -0.011656227521598339,
 0.06292832642793655,
 -0.03287624567747116,
 -0.09122596681118011,
 -0.03117532841861248,
 0.05269956216216087,
 0.04703476279973984,
 -0.0842030718922615,
 -0.030056225135922432,
 -0.020744914188981056,
